# GEM Strategy Tutorial

This notebook demonstrates how to use the improved GEM (Global Equities Momentum) strategy implementation.

## What is GEM?

The Global Equities Momentum strategy, developed by Gary Antonacci, combines:
- **Absolute Momentum**: Trend-following to avoid bear markets
- **Relative Momentum**: Cross-sectional comparison to select the best performing asset

The strategy makes monthly allocation decisions based on 12-month momentum indicators.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import GEM strategy components
from gem import GemConfig, GlobalEquitiesMomentum
from gem.strategy.backtest import Backtester
from gem.analysis.metrics import PerformanceMetrics
from gem.analysis.visualization import Visualizer

## 1. Configuration Setup

First, let's set up our configuration with custom parameters:

In [ ]:
# Create configuration
config = GemConfig()

# Customize strategy parameters
config.strategy.lookback_months = 12
config.strategy.transaction_cost = 0.001  # 0.1%
config.backtest.initial_capital = 10000
config.backtest.benchmark_symbol = "SPY"

# Display configuration
print("Strategy Configuration:")
print(f"Lookback Period: {config.strategy.lookback_months} months")
print(f"Transaction Cost: {config.strategy.transaction_cost:.3%}")
print(f"Initial Capital: ${config.backtest.initial_capital:,.2f}")
print(f"Benchmark: {config.backtest.benchmark_symbol}")

print("\nAsset Universe:")
for key, asset in config.assets.items():
    print(f"  {key}: {asset.symbol} - {asset.name}")

## 2. Initialize Strategy and Fetch Data

Let's initialize the strategy and fetch historical data:

In [ ]:
# Initialize strategy
strategy = GlobalEquitiesMomentum(config)

# Set date range for analysis
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 5)  # 5 years of data

print(f"Fetching data from {start_date.date()} to {end_date.date()}")

# Fetch data
strategy.fetch_data(start_date, end_date)

print("Data fetching completed!")
print(f"Available assets: {list(strategy.returns_data.keys())}")

## 3. Current Allocation Recommendation

Let's see what the strategy recommends for today:

In [ ]:
# Get current allocation
current_date = datetime.now()
current_allocation = strategy.calculate_strategy_allocation(current_date)

print(f"🎯 Current Allocation Recommendation ({current_allocation.date.date()})")
print(f"Signal: {current_allocation.signal.value.upper()}")
print(f"Primary Asset: {current_allocation.primary_asset}")

print("\n📊 Asset Allocation:")
for asset, weight in current_allocation.allocation.items():
    asset_name = config.assets[asset].name
    print(f"  {asset_name}: {weight:.1%}")

print("\n⚡ Momentum Scores:")
for asset, score in current_allocation.momentum_scores.items():
    asset_name = config.assets[asset].name
    print(f"  {asset_name}: {score:.2%}")

## 4. Strategy Backtest

Now let's run a comprehensive backtest:

In [ ]:
# Initialize backtester
backtester = Backtester(config)

print("Running backtest...")
backtest_results = backtester.run_backtest(
    start_date=start_date,
    end_date=end_date,
    initial_capital=config.backtest.initial_capital
)

print("Backtest completed!")
print(f"Backtest period: {len(backtest_results)} observations")
print(f"Final portfolio value: ${backtest_results['total_value'].iloc[-1]:,.2f}")

## 5. Performance Analysis

Let's analyze the performance metrics:

In [ ]:
# Get performance metrics
metrics = backtester.get_performance_metrics()

print("📊 PERFORMANCE METRICS")
print("=" * 50)

# Format and display key metrics
metric_names = {
    'total_return': 'Total Return',
    'annualized_return': 'Annualized Return',
    'volatility': 'Volatility',
    'sharpe_ratio': 'Sharpe Ratio',
    'max_drawdown': 'Maximum Drawdown',
    'win_rate': 'Win Rate',
    'final_portfolio_value': 'Final Value'
}

for key, name in metric_names.items():
    if key in metrics:
        value = metrics[key]
        if 'return' in key or 'drawdown' in key or 'rate' in key:
            formatted_value = f"{value:.2%}"
        elif 'ratio' in key:
            formatted_value = f"{value:.2f}"
        elif 'value' in key:
            formatted_value = f"${value:,.2f}"
        else:
            formatted_value = f"{value:.4f}"
        
        print(f"{name:.<30} {formatted_value}")

## 6. Benchmark Comparison

Let's compare our strategy against the benchmark:

In [ ]:
if 'benchmark_return' in backtest_results.columns:
    print("📈 BENCHMARK COMPARISON")
    print("=" * 40)
    
    strategy_total = metrics['total_return']
    benchmark_total = metrics.get('benchmark_total_return', 0)
    
    print(f"Strategy Total Return: {strategy_total:.2%}")
    print(f"Benchmark Total Return: {benchmark_total:.2%}")
    print(f"Excess Return: {strategy_total - benchmark_total:.2%}")
    
    if 'alpha' in metrics:
        print(f"Alpha: {metrics['alpha']:.2%}")
    if 'information_ratio' in metrics:
        print(f"Information Ratio: {metrics['information_ratio']:.2f}")
else:
    print("Benchmark comparison not available")

## 7. Visualization

Let's create some visualizations to better understand the strategy performance:

In [ ]:
# Initialize visualizer
visualizer = Visualizer(config)

# Get strategy returns
strategy_returns = backtest_results['portfolio_return'].dropna()
benchmark_returns = backtest_results.get('benchmark_return')

# Create cumulative returns chart
returns_data = {'GEM Strategy': strategy_returns}
if benchmark_returns is not None:
    returns_data['Benchmark (SPY)'] = benchmark_returns.dropna()

cumulative_returns_fig = visualizer.plot_cumulative_returns(
    returns_data,
    title="GEM Strategy vs Benchmark - Cumulative Returns",
    show_drawdown=True
)

cumulative_returns_fig.show()

In [ ]:
# Create allocation timeline
allocation_df = strategy.run_strategy(start_date, end_date)

allocation_fig = visualizer.plot_allocation_timeline(
    allocation_df,
    title="Asset Allocation Over Time"
)

allocation_fig.show()

In [ ]:
# Create rolling metrics chart
rolling_fig = visualizer.plot_rolling_metrics(
    strategy_returns,
    metrics=['return', 'volatility', 'sharpe'],
    window=12,  # 12-month rolling window
    title="Rolling Performance Metrics (12-Month Window)"
)

rolling_fig.show()

## 8. Risk Analysis

Let's perform a detailed risk analysis:

In [ ]:
# Initialize performance metrics calculator
perf_metrics = PerformanceMetrics(risk_free_rate=config.backtest.risk_free_rate)

# Calculate comprehensive risk metrics
risk_metrics = perf_metrics.calculate_risk_metrics(strategy_returns)

print("🚨 RISK ANALYSIS")
print("=" * 30)

print(f"Maximum Drawdown: {risk_metrics['max_drawdown']:.2%}")
print(f"Average Drawdown: {risk_metrics['avg_drawdown']:.2%}")
print(f"VaR (95%): {risk_metrics['value_at_risk']['95%']:.2%}")
print(f"CVaR (95%): {risk_metrics['conditional_var']['95%']:.2%}")
print(f"Calmar Ratio: {risk_metrics['calmar_ratio']:.2f}")
print(f"Skewness: {risk_metrics['skewness']:.2f}")
print(f"Kurtosis: {risk_metrics['kurtosis']:.2f}")

In [ ]:
# Create monthly returns heatmap
heatmap_fig = visualizer.plot_monthly_returns_heatmap(
    strategy_returns,
    title="Monthly Returns Heatmap"
)

heatmap_fig.show()

## 9. Strategy Summary

Let's get a summary of the strategy decisions:

In [ ]:
# Get strategy performance summary
strategy_summary = strategy.get_performance_summary()

print("📋 STRATEGY SUMMARY")
print("=" * 35)

print(f"Total Decisions: {strategy_summary['total_decisions']}")
print(f"Bullish Signals: {strategy_summary['bullish_signals']} ({strategy_summary['bullish_percentage']:.1f}%)")
print(f"Bearish Signals: {strategy_summary['bearish_signals']}")

print("\n🎯 Asset Selection Frequency:")
for asset, count in strategy_summary['asset_counts'].items():
    asset_name = config.assets[asset].name
    percentage = (count / strategy_summary['total_decisions']) * 100
    print(f"  {asset_name}: {count} times ({percentage:.1f}%)")

## 10. Export Results

Finally, let's export our results for further analysis:

In [ ]:
# Export backtest results
output_file = config.output_directory / "gem_backtest_results.csv"
backtest_results.to_csv(output_file)
print(f"📄 Backtest results saved to: {output_file}")

# Export allocation decisions
allocation_file = config.output_directory / "gem_allocations.csv"
allocation_df.to_csv(allocation_file)
print(f"📄 Allocation decisions saved to: {allocation_file}")

# Save performance dashboard
dashboard_fig = visualizer.create_performance_dashboard(
    strategy_returns=strategy_returns,
    benchmark_returns=benchmark_returns,
    allocation_df=allocation_df
)

visualizer.save_figure(dashboard_fig, "gem_performance_dashboard", "html")
print(f"📊 Performance dashboard saved to: {config.output_directory / 'gem_performance_dashboard.html'}")

## Conclusion

This tutorial demonstrated the key features of the improved GEM strategy implementation:

- **Easy Configuration**: Customizable parameters for different scenarios
- **Robust Data Handling**: Automatic data fetching with error handling
- **Comprehensive Analysis**: Detailed performance and risk metrics
- **Professional Visualization**: Interactive charts for better insights
- **Export Capabilities**: Easy data export for further analysis

The GEM strategy provides a systematic approach to momentum investing with built-in risk management through defensive asset allocation during unfavorable market conditions.

### Next Steps

1. **Experiment with Parameters**: Try different lookback periods and transaction costs
2. **Add Custom Assets**: Include additional ETFs or asset classes
3. **Extend the Analysis**: Compare with other momentum strategies
4. **Real-time Implementation**: Use the CLI for ongoing portfolio management

---

**⚠️ Disclaimer**: This is for educational purposes only. Past performance does not guarantee future results. Please consult with a financial advisor before making investment decisions.